In [12]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings('ignore', category=UserWarning)

In [13]:
# Custom layer handling for DepthwiseConv2D
class CustomDepthwiseConv2D(tf.keras.layers.DepthwiseConv2D):
    def __init__(self, *args, **kwargs):
        # Remove the 'groups' argument if present
        kwargs.pop('groups', None)
        super().__init__(*args, **kwargs)

# Register the custom layer
tf.keras.utils.get_custom_objects()['expanded_conv_depthwise'] = CustomDepthwiseConv2D

In [14]:
def load_deepfake_model(model_path):
    try:
        model = load_model(model_path, 
                           custom_objects={
                               'DepthwiseConv2D': CustomDepthwiseConv2D
                           }, 
                           compile=False)
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

In [15]:
def deepfake_detection(model_path, image_path):
    """
    Perform deepfake detection on a single image
    
    Args:
        model_path (str): Path to the .h5 model file
        image_path (str): Path to the image file
    """
    # Load the model
    model = load_deepfake_model(model_path)
    if model is None:
        print("Model could not be loaded. Exiting.")
        return

    # Define class names (modify as per your model's output)
    class_names = ['Real', 'Fake']  # Adjust based on your actual classes

    # Prepare data array
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

    try:
        # Open and prepare image
        image = Image.open(image_path).convert('RGB')
        
        # Resize and crop image
        size = (224, 224)
        image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
        
        # Convert to numpy array
        image_array = np.asarray(image)
        
        # Normalize the image
        normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
        
        # Load the image into the array
        data[0] = normalized_image_array
        
        # Predict
        prediction = model.predict(data)
        index = np.argmax(prediction)
        class_name = class_names[index]
        confidence_score = prediction[0][index]
        
        # Display results
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(image)
        plt.title('Original Image')
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.bar(class_names, prediction[0])
        plt.title(f'Prediction: {class_name}\nConfidence: {confidence_score:.2f}')
        plt.ylabel('Probability')
        
        plt.tight_layout()
        plt.show()
        
        # Print results
        print('Predicted Class:', class_name)
        print('Confidence Score:', confidence_score)
    
    except FileNotFoundError:
        print("Image not found. Please check the file path.")
    except Exception as e:
        print(f"Error processing image: {e}")


In [ ]:
# Usage example
if __name__ == "__main__":
    MODEL_PATH = 'deepfakemodel.h5'
    INPUT_FOLDER = './inputs'

    # Iterate through all .jpg files in the ./inputs folder
    for image_file in os.listdir(INPUT_FOLDER):
        if image_file.endswith('.jpg'):
            image_path = os.path.join(INPUT_FOLDER, image_file)
            print(f"Processing {image_file}...")
            deepfake_detection(MODEL_PATH, image_path)

Processing download-_1_.jpg...
